In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import chex
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange, tqdm
from functools import partial
from JAxtar.bgpq import HashTableHeapValue, BGPQ

/tmp/ipykernel_2880586/1118636839.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange, tqdm


In [3]:
max_size = int(1e5)
group_size = 1000
filled_sample = 1000

In [4]:
heap = BGPQ.make_heap(max_size, group_size, HashTableHeapValue)
insert = jax.jit(heap.insert)
delete_mins = jax.jit(heap.delete_mins)

2024-07-31 01:02:16.282788: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [5]:
heap_key = jax.random.uniform(jax.random.PRNGKey(0), shape=(filled_sample,), minval=0, maxval=10)
value = jax.vmap(HashTableHeapValue.default)(jnp.arange(filled_sample))
make_batch = BGPQ.make_batched(heap_key, value, group_size)

In [6]:
pbar = tqdm(total=int(max_size))
for i in range(0, max_size - 1, filled_sample):
    heap_key = jax.random.uniform(jax.random.PRNGKey(i), shape=(filled_sample,), minval=0, maxval=10)
    value = jax.vmap(HashTableHeapValue.default)(jnp.arange(filled_sample))
    block_key, block_val = BGPQ.make_batched(heap_key, value, group_size)
    heap = insert(heap, block_key[0], block_val[0])
    pbar.update(filled_sample)
pbar.close()

100%|██████████| 100000/100000 [00:02<00:00, 44248.01it/s]


In [7]:
pbar = tqdm(total=int(heap.size))
last_min = jnp.inf
last_max = -jnp.inf
while heap.size > 0:
    heap, min_key, min_val = delete_mins(heap)
    minimum = jnp.min(min_key)
    maximum = jnp.max(min_key)
    #print(minimum, maximum)
    if minimum < last_max:
        print("Error last_max", last_max, "minimum", minimum)
        print("Not sorted")
        break
    last_min = minimum
    last_max = maximum
    pbar.update(group_size)
pbar.close()

100%|██████████| 100000/100000 [00:02<00:00, 44571.00it/s]


In [8]:
print(heap.key_buffer)

[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf in